In [ ]:
import pandas as pd
from databaker.framework import *
from databakerUtils.writers import v4Writer
import requests

input_file = 'Headline Estimates A17M18.xls'
output_file = 'v4_Wellbeing_Estimate_18.csv'

tabs = loadxlstabs(input_file)

tabsThresholds = [tab for tab in tabs if "Thresholds" in tab.name]
tabsMeans = [tab for tab in tabs if "Means" in tab.name]

keyColumn = {'Thresholds':'O','Means':'A'}

def keyLocationCheck(tabs,location,tabName):
    '''
    Check to make sure key column is in 'correct' place
    '''
    for tab in tabs:
        key = tab.excel_ref(location).filter(contains_string("Key"))
        if len(key) != 1:
            raise Exception(
                "Key in {} tabs is in the wrong place"
                        "\n\nShould be in column {}".format(tabName,location)
                        )

keyLocationCheck(tabsThresholds,keyColumn['Thresholds'],'thresholds')
keyLocationCheck(tabsMeans,keyColumn['Means'],'means')

#'Thresholds' tabs first
conversionsegments = []

for tab in tabsThresholds:
    
    observationsWithKey = tab.excel_ref("C8").expand(RIGHT).expand(DOWN).is_not_blank().is_not_whitespace()
    key = tab.excel_ref(keyColumn['Thresholds']).filter(contains_string("Key")).expand(RIGHT).expand(DOWN)
    observations = observationsWithKey - key
    
    time = tab.excel_ref("C5").expand(RIGHT).is_not_blank().is_not_whitespace()
    
    geography = tab.excel_ref("A8").expand(DOWN).is_not_blank().is_not_whitespace()
    geographyLabels = tab.excel_ref("B8").expand(DOWN).is_not_blank().is_not_whitespace()
    
    indicators = tab.excel_ref("C7").expand(RIGHT).is_not_blank().is_not_whitespace()
    
    measures = tab.name.split("-")[0]
    
    dimensions = [
            HDim(time,TIME,CLOSEST,LEFT),
            HDim(geography,GEOG,DIRECTLY,LEFT),
            HDim(geographyLabels,"GeographyLabels",DIRECTLY,LEFT),
            HDimConst("allmeasuresofwellbeing",measures),
            HDim(indicators,"estimate",DIRECTLY,ABOVE)            
            ]
    
    conversionsegment = ConversionSegment(tab,dimensions,observations).topandas()
    conversionsegments.append(conversionsegment)
    
thresholdsData = pd.concat(conversionsegments)
thresholdsDF = v4Writer(output_file,thresholdsData,asFrame=True)

#'Means' tabs second
conversionsegments=[]
for tab in tabsMeans:
    
    observationsWithKey = tab.excel_ref("C8").expand(RIGHT).expand(DOWN).is_not_blank().is_not_whitespace()
    key = tab.excel_ref(keyColumn['Means']).filter(contains_string("Key")).expand(RIGHT).expand(DOWN)
    junk = tab.excel_ref("D").filter(contains_string("..")).expand(LEFT).expand(RIGHT)
    observations = observationsWithKey - key - junk
    
    time = tab.excel_ref("C6").expand(RIGHT).is_not_blank().is_not_whitespace()
    
    geography = tab.excel_ref("A8").expand(DOWN).is_not_blank().is_not_whitespace()
    geographyLabels = tab.excel_ref("B8").expand(DOWN).is_not_blank().is_not_whitespace()
    
    measures = tab.name.split("-")[0]
    
    indicators = "Average (mean)"
    
    dimensions = [
            HDim(time,TIME,DIRECTLY,ABOVE),
            HDim(geography,GEOG,DIRECTLY,LEFT),
            HDim(geographyLabels,"GeographyLabels",DIRECTLY,LEFT),
            HDimConst("allmeasuresofwellbeing",measures),
            HDimConst("estimate",indicators)            
            ]
    
    conversionsegment = ConversionSegment(tab,dimensions,observations).topandas()
    conversionsegments.append(conversionsegment)
    
meansData = pd.concat(conversionsegments)
meansDF = v4Writer(output_file,meansData,asFrame=True)

df = pd.concat([thresholdsDF,meansDF])  

'''Post Processing'''

def estimateLookup(value):
    value = value.strip()
    lookup={
            "9-10":"Very good",
            "0-1":"Very good",
            "7-8":"Good",
            "2-3":"Good",
            "5-6":"Fair",
            "5-4":"Fair",
            "4-5":"Fair",
            "0-4":"Poor",
            "6-10":"Poor",
            "Average (mean)":"Average (mean)"
            }
    return lookup[value]

#importing geography labes from api
url = 'https://api.cmd-dev.onsdigital.co.uk/v1/code-lists/admin-geography/editions/one-off/codes'
r = requests.get(url)
adminGeogDict = r.json()
geogDict = {}
for item in adminGeogDict['items']:
    geogDict.update({item['id']:item['label']})

#function to return geography labels from codes
def geogLookup(value):
    return geogDict[value]

colNames = {
        'estimate_codelist':'wellbeing-estimate',
        'allmeasuresofwellbeing_codelist':'wellbeing-measureofwellbeing',
        'Geography':'geography',
        'Geography_codelist':'admin-geography',
        'Time_codelist':'yyyy-yy',
        'Time':'time',
        'V4_0':'V4_1'
        }

df['Time'] = df['Time'].apply(lambda x:x.strip("*"))
df['Time_codelist'] = df['Time'].apply(lambda x:x.replace("/","-"))
df['Time'] = df['Time_codelist']

df['Geography'] = df['Geography_codelist'].apply(geogLookup)

df = df.drop(['GeographyLabels','GeographyLabels_codelist'],axis=1)

df['estimate'] =df['estimate'].apply(estimateLookup)
df['estimate_codelist'] = df['estimate'].apply(lambda x:x.lower()\
                          .strip()\
                          .replace("average (mean)","average mean")\
                          .replace(" ","-"))

df['allmeasuresofwellbeing'] = df['allmeasuresofwellbeing'].apply(lambda x:x.replace("Happy","Happiness")\
                               .strip())
df['allmeasuresofwellbeing_codelist'] = df['allmeasuresofwellbeing'].apply(lambda x:x.lower()\
                                        .replace(" ","-"))

df = df.rename(columns=colNames)

df.to_csv(output_file,index=False)